In [128]:
from bs4 import BeautifulSoup
import re
import requests

<h1> Scraping our data</h1>
<h3> First thing we need to do to start the project would be to gather our data</h3>
<p> We would like to find a reliable website of a very known fighting organization , this way we can trust the data since it is an official fighting organization </p>
<p>We are going to scrape ESPN official website for getting our data </p>
<a href="http://www.espn.com/mma/fighters">  Link to ESPN's website where we can observe the fighter list </a>

In [129]:
user_agent = {'User-agent': 'Mozilla/5.0'}

<h3> First we would like to have a list with links for all pages of of the fither list </h3>
<h3> first page will contain fithers with name started with letter A</h3>
<h3> first page will contain fithers with name started with letter Z</h3>

In [130]:
url1="http://www.espn.com/mma/fighters"
response1 = requests.get(url1,headers=user_agent)
pagesAToZ=[]
# we would like to add the first page (the page that contains fighters with letter A)
pagesAToZ.append(url1)
soup1 = BeautifulSoup(response1.content, "html.parser")
div_t=soup1.find("div",attrs={"class":"mod-content"})
hred_t=div_t.findAll("a")
for t in hred_t :
    pagesAToZ.append(url1+t['href'])
print(pagesAToZ)

['http://www.espn.com/mma/fighters', 'http://www.espn.com/mma/fighters?search=b', 'http://www.espn.com/mma/fighters?search=c', 'http://www.espn.com/mma/fighters?search=d', 'http://www.espn.com/mma/fighters?search=e', 'http://www.espn.com/mma/fighters?search=f', 'http://www.espn.com/mma/fighters?search=g', 'http://www.espn.com/mma/fighters?search=h', 'http://www.espn.com/mma/fighters?search=i', 'http://www.espn.com/mma/fighters?search=j', 'http://www.espn.com/mma/fighters?search=k', 'http://www.espn.com/mma/fighters?search=l', 'http://www.espn.com/mma/fighters?search=m', 'http://www.espn.com/mma/fighters?search=n', 'http://www.espn.com/mma/fighters?search=o', 'http://www.espn.com/mma/fighters?search=p', 'http://www.espn.com/mma/fighters?search=q', 'http://www.espn.com/mma/fighters?search=r', 'http://www.espn.com/mma/fighters?search=s', 'http://www.espn.com/mma/fighters?search=t', 'http://www.espn.com/mma/fighters?search=u', 'http://www.espn.com/mma/fighters?search=v', 'http://www.espn.c

<h3> Another function we would like to writw is a functions that scrapes the most inner page that contain previews fights stats , we would like to get a path as an argument</h3>

In [194]:
def scrapeinnerPageForFightStats(path):
    wins =""
    loses =""
    decision = ""
    winByKo = ""
    loseByKo= ""
    winbySubmission=""
    loseBySubmission=""
    page_link='http://www.espn.com'+path
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    regex_winsLosesDecision = "W-L-D(\d)-(\d)-(\d)"
    regex_ko="\(T\)KO(\d)-(\d)"
    regex_sub="SUB(\d)-(\d)"
    ul_t = soup.find("ul", attrs={"class":"StatBlock__Content flex list ph4 pv3 justify-between"}) 
    li_list=ul_t.findAll("li")
    for li in li_list:
       text=li.text
       print(text)
       winLoseDecStats_regex_res=re.search(regex_winsLosesDecision , text)
       koStats_regex_res=re.search(regex_ko , text)
       subStats_regex_res=re.search(regex_sub , text)
       if(winLoseDecStats_regex_res):
           wins=winLoseDecStats_regex_res.group(1)
           loses=winLoseDecStats_regex_res.group(2)
           decision=winLoseDecStats_regex_res.group(3)
           print(winLoseDecStats_regex_res)
       elif(koStats_regex_res):
           winByKo=koStats_regex_res.group(1)
           loseByKo=koStats_regex_res.group(2)
           print(koStats_regex_res)
       elif(subStats_regex_res):
           winbySubmission=subStats_regex_res.group(1)
           loseBySubmission=subStats_regex_res.group(2)
           print(subStats_regex_res)

    print(wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission)
    return wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission


In [195]:
scrapeinnerPageForFightStats('/mma/fighter/_/id/3043549/niina-aaltonen')

W-L-D0-1-0
<re.Match object; span=(0, 10), match='W-L-D0-1-0'>
(T)KO0-0
<re.Match object; span=(0, 8), match='(T)KO0-0'>
SUB0-1
<re.Match object; span=(0, 6), match='SUB0-1'>
0 1 0 0 0 0 1


('0', '1', '0', '0', '0', '0', '1')

<h3> Cool , We have a list of links to each page we want to scrape ! </h3>
<h4>Now we would like to build a function for scraping each page </h4>

<p>
/The structure of the page containes a long table with two columns . <br>
First Column - Name <br>
Sec Column - Country <br>

the Name is clickable and redirect us to another page with more details of the fighter
 </p>

In [133]:
def scrapePage(page_link):
# get list of table rows
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    table = soup.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    # we would like to remove the first two rows bc it doesnt contain any data
    del rows[0]
    del rows[0]
    for r in rows :
        cells = r.findAll(lambda tag: tag.name=='td')
        if(cells == None or (not cells)):
            continue 
        full_name = cells[0].find("a").text
        full_name=full_name.split(", ")
        first_name=full_name[0]
        last_name=full_name[1]
        country = cells[0].find("a")["href"]
        link_to_bio=cells[1].text
        if(first_name == None or last_name == None):
            continue
        #  pass link to get : wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission , reach from sub page 
        print(first_name , last_name)
        print(country)
        print(link_to_bio)
        print("============================================================================")



    

In [134]:
scrapePage('http://www.espn.com/mma/fighters')

Aaltonen Niina
/mma/fighter/_/id/3043549/niina-aaltonen

Aarab Soufyan
/mma/fighter/_/id/4898183/soufyan-aarab

Aaron Tom
/mma/fighter/_/id/2504991/tom-aaron
USA
Aaron Tommy
/mma/fighter/_/id/4239497/tommy-aaron
ESP
Aarons Joshua
/mma/fighter/_/id/3088828/joshua-aarons
ESP
Aarts Mike
/mma/fighter/_/id/3089919/mike-aarts
ESP
Abada Zyad
/mma/fighter/_/id/2511451/zyad-abada
ESP
Abadzhyan Kadzhik
/mma/fighter/_/id/2966179/kadzhik-abadzhyan
ESP
Abaev Imran
/mma/fighter/_/id/4401351/imran-abaev
RUS
Abafo Randen
/mma/fighter/_/id/4683514/randen-abafo
RUS
Abajian Jason
/mma/fighter/_/id/2501542/jason-abajian
RUS
Abanokov Soslan
/mma/fighter/_/id/3128222/soslan-abanokov
USA
Abargel Ariel
/mma/fighter/_/id/3149709/ariel-abargel
USA
Abarquez Marcus
/mma/fighter/_/id/3960654/marcus-abarquez
USA
Abasolo Eddie
/mma/fighter/_/id/4891258/eddie-abasolo
USA
Abasov Bagautdin
/mma/fighter/_/id/2999935/bagautdin-abasov
RUS
Abasov Emil
/mma/fighter/_/id/3961806/emil-abasov
RUS
Abasov Islam
/mma/fighter/_/id

KeyboardInterrupt: 